---
title: Tilt Corrected BF STEM
authors: [gvarnavides]
date: 2025-02-04
---

In [4]:
%matplotlib widget
import py4DSTEM

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from IPython.display import display
import ipywidgets
plt.rcParams['text.color']='white'
plt.rcParams['xtick.labelcolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.labelcolor'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'white'

In [5]:
file_path = 'data/'
file_data = file_path + 'parallax_apoferritin_cropped_binned.h5'

dataset = py4DSTEM.read(file_data)
dataset

DataCube( A 4-dimensional array of shape (24, 48, 32, 32) called 'datacube',
          with dimensions:

              Rx = [0.0,10.666666666666666,21.333333333333332,...] A
              Ry = [0.0,10.666666666666666,21.333333333333332,...] A
              Qx = [0.0,0.3076170140269948,0.6152340280539896,...] mrad
              Qy = [0.0,0.3076170140269948,0.6152340280539896,...] mrad
)

In [6]:
def add_poisson_noise(dataset,electrons_per_area):
    area = dataset.calibration.R_pixel_size ** 2
    electrons_per_probe = electrons_per_area * area
    dc = dataset.copy()
    dc.data = np.random.poisson(dc.data * electrons_per_probe).clip(0).astype(np.uint16)
    return dc

In [7]:
noisy_dataset = add_poisson_noise(
    dataset,
    50
)

In [8]:
energy = 300e3
object_padding_px = (8,8)
edge_blend = 4

In [9]:
parallax = py4DSTEM.process.phase.Parallax(
    datacube=dataset,
    energy = energy,
    object_padding_px=object_padding_px,
    defocus=1.5e4,
).preprocess(
    edge_blend=edge_blend,
    plot_average_bf=False,
    force_rotation_angle_deg=-15,
)

# ordering = np.argsort(parallax._kr)
inds = parallax._xy_inds - parallax._region_of_interest_shape/2
mag = np.linalg.norm(inds,axis=1)
theta = np.arctan2(
    inds[:,1],
    inds[:,0]
)

ordering = np.lexsort((theta,mag)) 

In [10]:
noisy_parallax = py4DSTEM.process.phase.Parallax(
    datacube=noisy_dataset,
    energy = energy,
    object_padding_px=object_padding_px,
    defocus=1.5e4,
).preprocess(
    edge_blend=edge_blend,
    dp_mask=parallax._dp_mask,
    plot_average_bf=False,
    force_rotation_angle_deg=-15,
)

In [11]:
def specified_shifts_ang(
    ordering,
):
    shifts = np.zeros_like(parallax._xy_shifts)
    shifts[ordering] = parallax._xy_shifts[ordering] * np.array(parallax._scan_sampling)
    return shifts

def specified_shifts_scaled(
    ordering,
    scale_arrows = 1,
):
    shifts = np.zeros_like(parallax._xy_shifts)
    shifts[ordering] = parallax._xy_shifts[ordering] * scale_arrows * np.array(parallax._reciprocal_sampling)
    return shifts

In [12]:
index = 0

In [16]:
dpi = 72
with plt.ioff():
    fig = plt.figure(figsize=(600/dpi,225/dpi),dpi=dpi)

gs = GridSpec(2, 6, figure=fig, wspace=0, hspace=0)
ax_scatter = fig.add_subplot(gs[:2,:2])
ax_scatter.patch.set_alpha(0)

ax_inf = fig.add_subplot(gs[0,2:4])
ax_fin = fig.add_subplot(gs[1,2:4])

ax_ibf = fig.add_subplot(gs[0,4:])
ax_par = fig.add_subplot(gs[1,4:])

parallax.show_shifts(
    specified_shifts_ang(ordering[:index]),
    scale_arrows=0.5,
    figax=(fig,ax_scatter),
)
scatter = ax_scatter.collections[0]
scatter.set_color((255/255, 249/255, 148/255))
dot = ax_scatter.scatter(0,0,color='white')
ax_scatter.set_title("planewave shifts")

inf_unshifted = parallax._crop_padded_object(
    parallax._stack_BF_unshifted[ordering[index]]
)

inf_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(inf_unshifted,normalize=True)
im_inf_unshifted = ax_inf.imshow(inf_unshifted_scaled,cmap='gray')
ax_inf.set_title("shifted image")

fin_unshifted = parallax._crop_padded_object(
    noisy_parallax._stack_BF_unshifted[ordering[index]]
)

fin_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(fin_unshifted,normalize=True)
im_fin_unshifted = ax_fin.imshow(fin_unshifted_scaled,cmap='gray')
ax_fin.set_xlabel("noisy shifted image",fontsize=12)

ibf_unshifted = parallax._crop_padded_object(
    noisy_parallax._stack_BF_unshifted[ordering[:index+1]].mean(0)
)

ibf_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(ibf_unshifted,normalize=True)
im_ibf_unshifted = ax_ibf.imshow(ibf_unshifted_scaled,cmap='gray')
ax_ibf.set_title("incoherent sum")
ax_ibf.set_visible(False)

par_shifted = parallax._crop_padded_object(
    noisy_parallax._stack_BF_shifted[ordering[:index+1]].mean(0)
)

par_shifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(par_shifted,normalize=True)
im_par_shifted = ax_par.imshow(par_shifted_scaled,cmap='gray')
ax_par.set_xlabel("aligned sum",fontsize=12)
ax_par.set_visible(False)


bar = {
    'pixelsize':parallax._scan_sampling[1]/10,
    'pixelunits':'nm',
    "Nx":parallax._scan_shape[0],
    "Ny":parallax._scan_shape[1],
    "labelsize":8,
    "width":2,
    "length":5
}

for ax in [ax_inf,ax_fin,ax_ibf,ax_par]:
    py4DSTEM.visualize.add_scalebar(ax,bar)
    ax.set(xticks=[],yticks=[])

gs.tight_layout(fig)
fig.patch.set_alpha(0)
fig.canvas.resizable = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.toolbar_visible = False
fig.canvas.layout.width = '600px'
# fig.canvas.layout.height = "225px"
fig.canvas.toolbar_position = "bottom"
None

In [17]:
def update_scatter_plot(
    index,
):
    """ """
    indices = ordering[:index+1]
    last_index = ordering[index]
    
    v,u = specified_shifts_scaled(
        ordering[:index+1],
        scale_arrows=0.5
    ).T
    
    scatter.set_UVC(u,v)
    ky, kx = parallax._kxy[last_index]
    dot.set_offsets([kx,ky])
                       
    fig.canvas.draw_idle()
    return None

def update__plots(
    index,
):
    """ """
    indices = ordering[:index+1]
    last_index = ordering[index]
    
    inf_unshifted = parallax._crop_padded_object(
        parallax._stack_BF_unshifted[last_index]
    )
    
    inf_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(inf_unshifted,normalize=True)
    im_inf_unshifted.set_data(inf_unshifted_scaled)
    
    fin_unshifted = parallax._crop_padded_object(
        noisy_parallax._stack_BF_unshifted[last_index]
    )
    
    fin_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(fin_unshifted,normalize=True)
    im_fin_unshifted.set_data(fin_unshifted_scaled)
    
    ibf_unshifted = parallax._crop_padded_object(
        noisy_parallax._stack_BF_unshifted[indices].mean(0)
    )
    
    ibf_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(ibf_unshifted,normalize=True)
    im_ibf_unshifted.set_data(ibf_unshifted_scaled)
    
    par_shifted = parallax._crop_padded_object(
        noisy_parallax._stack_BF_shifted[indices].mean(0)
    )
    
    par_shifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(par_shifted,normalize=True)
    im_par_shifted.set_data(par_shifted_scaled)
    fig.canvas.draw_idle()
    return None

def update_both(change):
    """ """
    index = change["new"]
    update_scatter_plot(index)
    update__plots(index)
    return None

def toggle_visibility(change):
    """ """
    visibility = change["new"]
    ax_ibf.set_visible(visibility)
    ax_par.set_visible(visibility)
    fig.canvas.draw_idle()
    return None

half_layout = ipywidgets.Layout(width='300px',height='30px')
quarter_layout = ipywidgets.Layout(width='150px',height='30px')

style = {
    'description_width': 'initial',
}

toggle = ipywidgets.ToggleButton(
    value=False,
    description="align images",
    style=style,
    layout=quarter_layout,
)

play = ipywidgets.Play(
    value=0,
    min=0,
    max=parallax._num_bf_images-1,
    step=1,
    interval=250,
    show_repeat=False,
    style=style,
    layout=quarter_layout,
)

slider = ipywidgets.IntSlider(
    min=0,
    max=parallax._num_bf_images-1,
    step=1,
    layout=half_layout,
    style=style,
    description="tilted planewave index"
)

ipywidgets.jslink((play, 'value'), (slider, 'value'))
slider.observe(update_both,"value")
toggle.observe(toggle_visibility,"value")

In [18]:
#| label: app:parallax-alignment

ipywidgets.VBox(
    [
        ipywidgets.HBox([slider,toggle]),
        fig.canvas
    ],
    layout=ipywidgets.Layout(
        align_items="center"
    )
)